# Project 4

In this project, you will summarize and present your analysis from Projects 1-3.

### Intro: Write a problem Statement/ Specific Aim for this project

Answer: Predict the probability that a student will be admitted into graduate school using the undergraduate school prestige, GRE score, and GPA.

### Dataset:  Write up a description of your data and any cleaning that was completed

Answer: The data contains four variables - admit, GRE, GPA, and prestige.

Variable | Description | Type of Variable | DType 
---| ---| ---
admit | 0 = not admitted, 1 = admitted | categorical | integer
gre | candidate GRE score | continuous | float
gpa | candidate Grade Point Average | continuous | float
prestige | rank of undergraduate alma mater: 1 = highest, 4 = lowest | categorical | float

Initally, the data contained 400 observations. In cleaning the data, I dropped data points with missing values, leaving us with 397 observations.

### Demo: Provide a table that explains the data by admission status

Mean (STD) or counts by admission status for each variable 

| Not Admitted | Admitted
---| ---|---
GPA | mean(std)  | mean(std)
GRE |mean(std) | mean(std)
Prestige 1 | frequency (%) | frequency (%)
Prestige 2 | frequency (%) | frequency (%)
Prestige 3 |frequency (%) | frequency (%)
Prestige 4 |frequency (%) | frequency (%)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [3]:
df_raw = pd.read_csv("assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


In [4]:
df = pd.get_dummies(df, columns=['prestige'])
df.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [26]:
table = pd.DataFrame(data='NA', index=['GPA Mean','GPA STD','GRE Mean','GRE STD','Prestige 1','Prestige 2','Prestige 3','Prestige 4'], 
                  columns=['Not Admitted','Admitted'])

In [27]:
table.loc['GPA Mean','Not Admitted']=df['gpa'].groupby(df['admit']).mean()[0]
table.loc['GPA STD','Not Admitted']=df['gpa'].groupby(df['admit']).std()[0]
table.loc['GPA Mean','Admitted']=df['gpa'].groupby(df['admit']).mean()[1]
table.loc['GPA STD','Admitted']=df['gpa'].groupby(df['admit']).std()[1]

table.loc['GRE Mean','Not Admitted']=df['gre'].groupby(df['admit']).mean()[0]
table.loc['GRE STD','Not Admitted']=df['gre'].groupby(df['admit']).std()[0]
table.loc['GRE Mean','Admitted']=df['gre'].groupby(df['admit']).mean()[1]
table.loc['GRE STD','Admitted']=df['gre'].groupby(df['admit']).std()[1]

table.loc['Prestige 1','Not Admitted']=float(len(df[(df['prestige_1.0']==1)&(df['admit']==0)]))/len(df[df['prestige_1.0']==1])
table.loc['Prestige 2','Not Admitted']=float(len(df[(df['prestige_2.0']==1)&(df['admit']==0)]))/len(df[df['prestige_2.0']==1])
table.loc['Prestige 3','Not Admitted']=float(len(df[(df['prestige_3.0']==1)&(df['admit']==0)]))/len(df[df['prestige_3.0']==1])
table.loc['Prestige 4','Not Admitted']=float(len(df[(df['prestige_4.0']==1)&(df['admit']==0)]))/len(df[df['prestige_4.0']==1])

table.loc['Prestige 1','Admitted']= 1 - table.loc['Prestige 1','Not Admitted']
table.loc['Prestige 2','Admitted']= 1 - table.loc['Prestige 2','Not Admitted']
table.loc['Prestige 3','Admitted']= 1 - table.loc['Prestige 3','Not Admitted']
table.loc['Prestige 4','Admitted']= 1 - table.loc['Prestige 4','Not Admitted']

table

,Not Admitted,Admitted
GPA Mean,3.34716,3.48921
GPA STD,0.376355,0.371655
GRE Mean,573.579,618.571
GRE STD,116.053,109.257
Prestige 1,0.459016,0.540984
Prestige 2,0.641892,0.358108
Prestige 3,0.768595,0.231405
Prestige 4,0.820896,0.179104


### Methods: Write up the methods used in your analysis

Answer: After cleaning the data and adding dummy variables for prestige, I added a constant term for the logistic regression and defined the training columns. I then used the Logit function in statsmodels to fit the model and predict admission. 


As a way of evaluating our classifier, we used a helper Cartesian function to generate every logical combination of input values from the dataset. This allowed us to see how the predicted probability of admission changes across different variables. 

### Results: Write up your results

Answer: The p-values for all coefficients were <0.05, therefore all variables were statistically significant. GRE and GPA are positively correlated with the probability of admission. There is also an inverse relationship between prestige ranking and probability of admission.

### Visuals: Provide a table or visualization of these results

In [28]:
df = df_raw.dropna() 
df = pd.get_dummies(df, columns=['prestige'], drop_first=True)

# manually add the intercept
df['intercept'] = 1.0

train_cols = ['gre','gpa','prestige_2.0','prestige_3.0','prestige_4.0','intercept']

model = sm.Logit(df['admit'], df[train_cols])

results = model.fit()
print results.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 06 Mar 2017   Pseudo R-squ.:                 0.08166
Time:                        22:32:32   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793

### Discussion: Write up your discussion and future steps

Answer: GRE and GPA scores correlate positively with the probability that a student is admitted into graduate school. Prestige is negatively correlated, with the magnitude increasing the lower the prestige ranking.

For future steps, we could try different models (eg. random forest) and evaluate performance to see if there exists a better prediction method.